In [ ]:
def twitter(names, counts, dates):
    
    consumer_key = 'Put your consumer_key here'
    consumer_secret = 'Put your consumer_secret here'
    access_token = 'Put your access_token here'
    access_token_secret = 'Put your access_token_secret here'
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    
    all_tweets = []
    error_names = []
    outtweets = [[]]
    T = True
    
    for i in range(len(names)):
        user_tweets = []
        
        try:
            try:
                cn = int(counts[i])
            except:
                cn = 20
                
            cn_new = cn
            if cn > 200:
                cn_new = 200
                
            tweets = api.user_timeline(screen_name = names[i], count=cn_new, include_rts = False, tweet_mode = 'extended')
            user_tweets.extend(tweets)
            oldest_id = tweets[-1].id
            cn_new = cn - 200
            
            if cn_new > 0:
                while cn_new > 0:
                    cn_new2 = cn_new
                    if cn_new > 200:
                        cn_new2 = 200
                    tweets = api.user_timeline(screen_name = names[i],
                                               count=cn_new2,
                                               include_rts = False,
                                               max_id = oldest_id - 1,
                                               tweet_mode = 'extended')
                    
                    if len(tweets) == 0:
                        break
                        
                    oldest_id = tweets[-1].id
                    user_tweets.extend(tweets)
                    cn_new = cn_new - 200
                    
            all_tweets.extend(user_tweets)
            outtweets = [[tweet.user.name, 
                          tweet.created_at.strftime("%m/%d/%Y"),
                          tweet.created_at.strftime("%H:%M:%S"),
                          tweet.favorite_count,
                          tweet.full_text.encode("utf-8").decode("utf-8")] 
                         for idx,tweet in enumerate(all_tweets)]
        except:
            T = False
            error_names.append(names[i])
            
    if outtweets == [[]]:
        df = pd.DataFrame(columns=["User", "Date", "Time", "Tweet"])
        
    elif len(outtweets) == 0:
        df = pd.DataFrame(columns=["User", "Date", "Time", "Tweet"])
        
    else:
        df = pd.DataFrame(outtweets,columns=["User", "Date", "Time", "Likes Number", "Tweet"])
        names = [i for i in df.User.unique()]
        df_new = pd.DataFrame()
        
        for u,d in zip(names,dates):
            df_s = df[df.User == u]
            df_s = df_s[pd.to_datetime(df_s['Date'])>=pd.to_datetime(d)]
            df_new = pd.concat([df_s, df_new])
            
        df = df_new
        
    return df, T, error_names

In [ ]:
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd
from tkcalendar import Calendar,DateEntry
from datetime import datetime
import tweepy

bg = 'rosybrown1'
entry_bg = 'white smoke'

root = Tk()

root.configure(background=bg)
root.title('twitterholic')
root.iconbitmap('twitter.ico')

root.geometry("500x400")
empty_label = Label(root, text = "", padx=20, pady=5, background=bg)
empty_label.pack()

twitter_image = Image.open("twitter.png")
twitter_image = twitter_image.resize((48, 38), Image.ANTIALIAS)
twitter_image = ImageTk.PhotoImage(twitter_image)
image = Label(image = twitter_image, padx=10, pady=10, bg=bg)
image.pack()

def main(Bool):
    global root, empty_label, image, user_name_label, user_name_entry, number_label, number_entry
    global date_label, calendar, user_names_list, numbers_list, dates_list, enter_button, df, v, empty_label2
    global repeat_label, repeat_button
    
    if Bool:
        repeat_label.destroy()
        repeat_button.destroy()
        empty_label2.destroy()

    root.geometry("500x400")

    user_name_text = "Please enter the user screen names you want to extract tweets from!"
    user_name_label = Label(root, text = user_name_text, padx=40, pady=20, background=bg)
    user_name_label.pack()

    user_name_entry = Entry(root, width=30, fg='black', bg=entry_bg)
    user_name_entry.insert(0, "Enter screen name here please!")
    user_name_entry.pack()

    text_number_label = "Please insert below the number of tweets you want to extract!"
    number_label = Label(root, text = text_number_label, padx=40, pady=20, background=bg)
    number_label.pack()

    number_entry = Entry(root, width=45, fg='black', bg=entry_bg)
    number_entry.insert(0, "Enter the number of tweets here!")
    number_entry.pack()

    date_label = Label(root, text = "Please choose the date to extract from!", padx=40, pady=20, background=bg)
    date_label.pack()

    year = int(datetime.today().strftime('%Y'))
    month = int(datetime.today().strftime('%m'))
    day = int(datetime.today().strftime('%d'))
    
    calendar = DateEntry(root,fg='white', background='black', width=30, year=year, month=month, day=day)
    calendar.pack()
    
    user_name_entry.bind("<Button-1>", user_name_entry_callback)
    number_entry.bind("<Button-1>", number_entry_callback)
    
    user_names_list = []
    numbers_list = []
    dates_list = []
    v = 0
    df = pd.DataFrame()
        
    empty_label2 = Label(root, text = "", padx=20, pady=5, background=bg)
    empty_label2.pack()

    enter_button = Button(root, text='Enter', command=ask_verif, bg='steelblue', fg='white', height = 2, width = 8)
    enter_button.pack()

def user_name_entry_callback(event):
    user_name_entry.delete(0, "end")
    
def number_entry_callback(event): 
    number_entry.delete(0, "end")
    
def save_file(df):
    global repeat_label, repeat_button, file_ready_label, save_button, image
    result = filedialog.asksaveasfilename()
    if result != '':
        filename = result + '.xlsx'
        df.to_excel(filename, index=False, encoding='utf-8')
        file_ready_label.destroy()
        save_button.destroy()
        repeat_label = Label(root, text = "Would you like to do another extraction?", padx=40, pady=20, background=bg)
        repeat_label.pack()
        repeat_button = Button(root, text='Repeat', command=lambda:main(True),
                               bg='steelblue', fg='white', height = 1, width = 8)
        repeat_button.pack()
        
def ask_verif():
    global root, empty_label, image, user_name_label, user_name_entry, number_label, number_entry
    global date_label, calendar, user_names_list, numbers_list, dates_list, enter_button, df, v, empty_label2
    global invalid_label, file_ready_label, save_button
    
    dates_list.append(calendar.get())
    user_names_list.append(user_name_entry.get())
    numbers_list.append(number_entry.get())
    
    user_name_entry.bind("<Button-1>", user_name_entry_callback)
    question_text = "Do you want to add another user name?\nPlease wait for a moment if you choose no!"
    response = messagebox.askquestion("", question_text)
    
    if response == 'no':
        df_new, T, error_names = twitter(user_names_list, numbers_list, dates_list)
        df = pd.concat([df,df_new])
        if T == False:
            n = len(error_names)
            if n == 1:
                if v != 0:
                    invalid_label.destroy()
                    
                root.geometry("500x520")
                invalid_text = str(n) + ' name is invalid: \n' + str(error_names[0]) + '\n Please retry the wrong name!'
                invalid_label = Label(root, text = invalid_text, padx=40, pady=20, background=bg)
                invalid_label.pack()
                user_names_list = []
                numbers_list = []
                dates_list = []
                v = 1
                
            else:
                if v != 0:
                    invalid_label.destroy()

                ch = ''
                for c in error_names:
                    ch += str(c) + '\n'
                root.geometry("500x600")
                invalid_text = str(n) + ' names are invalid: \n' + ch + '\n Please retry all the wrong names!'
                invalid_label = Label(root, text = invalid_text, padx=40, pady=20, background=bg)
                invalid_label.pack()
                user_names_list = []
                numbers_list = []
                dates_list = []
                v = 1
                
        else:
            user_name_entry.destroy()
            number_entry.destroy()
            enter_button.destroy()
            number_label.destroy()
            date_label.destroy()
            calendar.destroy()
            if v!=0:
                invalid_label.destroy()
            user_name_label.destroy()
            empty_label2.destroy()
            root.geometry("500x200")
            file_ready_text = "Your file is ready, please name and save it in your machine"
            file_ready_label = Label(root, text = file_ready_text, padx=40, pady=20, background=bg)
            file_ready_label.pack()
            save_button = Button(root, text = "Save file",
                                 command=lambda: save_file(df), bg='steelblue', fg='white', height = 1, width = 8)
            save_button.pack()

main(False)
    
close_button = Button(root, text = "Exit", command=root.destroy, bg='red4', fg='white', height = 1, width = 5)
close_button.pack(side=BOTTOM, anchor=E)

root.mainloop()